In [2]:
import os
import boto3

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

import sagemaker
from sagemaker import get_execution_role
import sagemaker_pyspark
from pyspark.sql.functions import *

role = get_execution_role()

# Configure Spark to use the SageMaker Spark dependency jars
jars = sagemaker_pyspark.classpath_jars()

classpath = ":".join(sagemaker_pyspark.classpath_jars())

# See the SageMaker Spark Github to learn how to connect to EMR from a notebook instance
spark = SparkSession.builder.config("spark.driver.extraClassPath", classpath)\
    .master("local[*]").getOrCreate()
    
spark

In [3]:
import os
import pandas as pd
import numpy as np
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
import pyspark.sql.functions as fn
from pyspark.sql.functions import col, udf

In [4]:
cdc2 = spark.read.csv('output.csv', header = True)

In [ ]:
cdc2 = spark.read.csv('s3://console.aws.amazon.com/s3/buckets/mrinal-ml-sagemaker/Project/output.csv', header = True)

In [ ]:
cdc2 = spark.read.csv('output.csv', header = True)

In [5]:
cdc2.printSchema()


root
 |-- resident_status: string (nullable = true)
 |-- education_1989_revision: string (nullable = true)
 |-- education_2003_revision: string (nullable = true)
 |-- education_reporting_flag: string (nullable = true)
 |-- month_of_death: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- detail_age_type: string (nullable = true)
 |-- detail_age: string (nullable = true)
 |-- age_substitution_flag: string (nullable = true)
 |-- age_recode_52: string (nullable = true)
 |-- age_recode_27: string (nullable = true)
 |-- age_recode_12: string (nullable = true)
 |-- infant_age_recode_22: string (nullable = true)
 |-- place_of_death_and_decedents_status: string (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- day_of_week_of_death: string (nullable = true)
 |-- current_data_year: string (nullable = true)
 |-- injury_at_work: string (nullable = true)
 |-- manner_of_death: string (nullable = true)
 |-- method_of_disposition: string (nullable = true)
 |-- autopsy

In [ ]:
cdc2.printSchema()

In [6]:
cdc2 = cdc2.drop('detail_age_type','detail_age', 'age_substitution_flag','age_recode_27', 'age_recode_12', 'infant_age_recode_22', 'icd_code_10th_revision')

In [7]:
cdc2 = cdc2.drop('detail_age_type','detail_age', 'age_substitution_flag','age_recode_27', 'age_recode_12', 'infant_age_recode_22', 'icd_code_10th_revision')

In [8]:
cdc2 = cdc2.drop('record_condition_20' , 'entity_condition_20', 'entity_condition_19', 'entity_condition_18', 'entity_condition_17', 'record_condition_19', 'record_condition_18', 'record_condition_16', 'record_condition_17','record_condition_15','record_condition_14','record_condition_13','record_condition_12','record_condition_11','record_condition_10','record_condition_9','record_condition_8','record_condition_7' , 'record_condition_6', 'record_condition_5', 'entity_condition_16', 'entity_condition_15','entity_condition_14', 'entity_condition_13', 'entity_condition_12', 'entity_condition_11', 'entity_condition_10', 'entity_condition_9','entity_condition_8', 'entity_condition_7', 'entity_condition_6', 'entity_condition_5')

In [9]:
cdc2.printSchema()

root
 |-- resident_status: string (nullable = true)
 |-- education_1989_revision: string (nullable = true)
 |-- education_2003_revision: string (nullable = true)
 |-- education_reporting_flag: string (nullable = true)
 |-- month_of_death: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age_recode_52: string (nullable = true)
 |-- place_of_death_and_decedents_status: string (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- day_of_week_of_death: string (nullable = true)
 |-- current_data_year: string (nullable = true)
 |-- injury_at_work: string (nullable = true)
 |-- manner_of_death: string (nullable = true)
 |-- method_of_disposition: string (nullable = true)
 |-- autopsy: string (nullable = true)
 |-- activity_code: string (nullable = true)
 |-- place_of_injury_for_causes_w00_y34_except_y06_and_y07_: string (nullable = true)
 |-- 358_cause_recode: string (nullable = true)
 |-- 113_cause_recode: string (nullable = true)
 |-- 130_infant_cause_recode: 

In [10]:

cdc = cdc.fillna({'place_of_injury_for_causes_w00_y34_except_y06_and_y07_' : '10'})
cdc = cdc.fillna({'130_infant_cause_recode' : '000'})
cdc = cdc.fillna({'activity_code' : '10'})
cdc = cdc.fillna({'manner_of_death' : '8'})
cdc = cdc.fillna({'Place_of_death_and_decedents_status': '7'})

NameError: name 'cdc' is not defined

In [11]:

cdc2 = cdc2.fillna({'place_of_injury_for_causes_w00_y34_except_y06_and_y07_' : '10'})
cdc2 = cdc2.fillna({'130_infant_cause_recode' : '000'})
cdc2 = cdc2.fillna({'activity_code' : '10'})
cdc2 = cdc2.fillna({'manner_of_death' : '8'})
cdc2 = cdc2.fillna({'Place_of_death_and_decedents_status': '7'})

In [12]:
cdc2 = cdc2.withColumn('method_of_disposition', regexp_replace('method_of_disposition', 'R' , 'O'))
cdc2 = cdc2.withColumn('method_of_disposition', regexp_replace('method_of_disposition', 'E' , 'O'))
cdc2 = cdc2.withColumn('method_of_disposition', regexp_replace('method_of_disposition', 'D' , 'O'))
cdc2 = cdc2.withColumn('method_of_disposition', regexp_replace('method_of_disposition', 'U' , 'O'))
cdc2 = cdc2.withColumn('Place_of_death_and_decedents_status', regexp_replace('Place_of_death_and_decedents_status', '9' , '7'))

In [13]:
cdc2 = cdc2.withColumn('Place_of_death_and_decedents_status', 
                   when(cdc2['Place_of_death_and_decedents_status']== 9 , 7).otherwise(cdc2['Place_of_death_and_decedents_status']))

In [14]:
cdc2.select('Place_of_death_and_decedents_status').distinct().show()

+-----------------------------------+
|Place_of_death_and_decedents_status|
+-----------------------------------+
|                                  7|
|                                  3|
|                                  5|
|                                  6|
|                                  1|
|                                  4|
|                                  2|
+-----------------------------------+



In [15]:
cdc2.select('place_of_injury_for_causes_w00_y34_except_y06_and_y07_').distinct().show()

+------------------------------------------------------+
|place_of_injury_for_causes_w00_y34_except_y06_and_y07_|
+------------------------------------------------------+
|                                                     7|
|                                                     3|
|                                                     8|
|                                                     0|
|                                                     5|
|                                                     6|
|                                                     9|
|                                                     1|
|                                                    10|
|                                                     4|
|                                                     2|
+------------------------------------------------------+



In [16]:
cdc2 = cdc2.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '00' , '1'))
cdc2 = cdc2.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '01' , '1'))
cdc2 = cdc2.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '02' , '1'))
cdc2 = cdc2.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '03' , '1'))
cdc2 = cdc2.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '04' , '1'))
cdc2 = cdc2.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '05' , '1'))
cdc2 = cdc2.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '06' , '1'))
cdc2 = cdc2.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '07' , '1'))
cdc2 = cdc2.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '08' , '1'))

cdc2 = cdc2.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '09' , '2'))
cdc2 = cdc2.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '10' , '2'))
cdc2 = cdc2.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '11' , '2'))
cdc2 = cdc2.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '12' , '3'))
cdc2 = cdc2.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '13' , '4'))
cdc2 = cdc2.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '14' , '4'))
cdc2 = cdc2.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '15' , '5'))
cdc2 = cdc2.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '16' , '6'))
cdc2 = cdc2.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '17' , '7'))
cdc2 = cdc2.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '99' , '9'))

In [17]:
cdc2 = cdc2.withColumn("education_2003_revision", coalesce(cdc2.education_2003_revision,cdc2.education_1989_revision))

In [18]:
cdc2 = cdc2.filter(cdc2.education_2003_revision.isNotNull())

In [19]:
cdc2.groupBy('education_2003_revision').count().orderBy('count', ascending=False).show()

+-----------------------+--------+
|education_2003_revision|   count|
+-----------------------+--------+
|                      3|11329932|
|                      1| 3805467|
|                      4| 3293354|
|                      2| 3226815|
|                      6| 2480719|
|                      5| 1144604|
|                      7| 1106055|
|                      9| 1024952|
|                      8|  308775|
+-----------------------+--------+



In [20]:
cdc2 = cdc2.drop('education_1989_revision', 'education_reporting_flag','bridged_race_flag', 'race_imputation_flag', 
               'hispanic_origin','entity_condition_1','entity_condition_2','entity_condition_3','entity_condition_4',
              'record_condition_1','record_condition_2','record_condition_3','record_condition_4')

In [21]:
cdc2.printSchema()

root
 |-- resident_status: string (nullable = true)
 |-- education_2003_revision: string (nullable = true)
 |-- month_of_death: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age_recode_52: string (nullable = true)
 |-- Place_of_death_and_decedents_status: string (nullable = false)
 |-- marital_status: string (nullable = true)
 |-- day_of_week_of_death: string (nullable = true)
 |-- current_data_year: string (nullable = true)
 |-- injury_at_work: string (nullable = true)
 |-- manner_of_death: string (nullable = false)
 |-- method_of_disposition: string (nullable = true)
 |-- autopsy: string (nullable = true)
 |-- activity_code: string (nullable = false)
 |-- place_of_injury_for_causes_w00_y34_except_y06_and_y07_: string (nullable = false)
 |-- 358_cause_recode: string (nullable = true)
 |-- 113_cause_recode: string (nullable = true)
 |-- 130_infant_cause_recode: string (nullable = false)
 |-- 39_cause_recode: string (nullable = true)
 |-- number_of_entity_axis_conditi

In [22]:
cdc2.groupBy('358_cause_recode').where(358_cause_recode==132).count().orderBy('count', ascending=False).show()

SyntaxError: invalid token (<ipython-input-22-c6471fcfcbfc>, line 1)

In [23]:
cdc3= cdc2.withcolumn('358_cause_recode').when(cdc2['358_cause_recode']== '407').count().orderBy('count', ascending=False).show()

AttributeError: 'DataFrame' object has no attribute 'withcolumn'

In [24]:
cdc2.groupBy('358_cause_recode').count().orderBy('count', ascending=False).show()

+----------------+-------+
|358_cause_recode|  count|
+----------------+-------+
|             215|2212520|
|             093|1733447|
|             211|1392415|
|             267|1367934|
|             175|1164777|
|             189| 921265|
|             159| 807571|
|             238| 755369|
|             214| 657654|
|             230| 607764|
|             257| 541840|
|             327| 494823|
|             228| 489941|
|             081| 468606|
|             104| 455641|
|             125| 439283|
|             088| 405728|
|             023| 399667|
|             207| 376585|
|             420| 349322|
+----------------+-------+
only showing top 20 rows



In [27]:
with open('2005_codes.json', 'r') as f_open:
    code_maps_2005 = json.load(f_open)


In [26]:
import json 

In [28]:
code_maps_2005

{'resident_status': {'1': 'RESIDENTS',
  '2': 'INTRASTATE NONRESIDENTS',
  '3': 'INTERSTATE NONRESIDENTS',
  '4': 'FOREIGN RESIDENTS'},
 'education_1989_revision': {'00': 'No formal education',
  '01-08': 'Years of elementary school',
  '09': '1 year of high school',
  '10': '2 years of high school',
  '11': '3 years of high school',
  '12': '4 years of high school',
  '13': '1 year of college',
  '14': '2 years of college',
  '15': '3 years of college',
  '16': '4 years of college',
  '17': '5 or more years of college',
  '99': 'Not stated'},
 'education_2003_revision': {'1': '8th grade or less',
  '2': '9 - 12th grade, no diploma',
  '3': 'high school graduate or GED completed',
  '4': 'some college credit, but no degree',
  '5': 'Associate degree',
  '6': 'Bachelor’s degree',
  '7': 'Master’s degree',
  '8': 'Doctorate or professional degree',
  '9': 'Unknown'},
 'education_reporting_flag': {'0': '1989 revision of education item on certificate',
  '1': '2003 revision of education it

In [29]:
with open('2005_codes.json', 'r') as f_open:
    code_maps_2005 = json.load(f_open)

In [30]:
cdc2['358_cause_recode'] = cdc2['358_cause_recode'].apply(
    lambda x: code_maps_2005["358_cause_recode"][str(x)])

TypeError: 'Column' object is not callable

In [31]:
cdc2.groupBy('month_of_death','sex').count().orderBy('sex', ascending=False).show()

+--------------+---+-------+
|month_of_death|sex|  count|
+--------------+---+-------+
|            12|  M|1237273|
|            02|  M|1147291|
|            01|  M|1269199|
|            09|  M|1081367|
|            10|  M|1147902|
|            04|  M|1145743|
|            06|  M|1087923|
|            08|  M|1107852|
|            07|  M|1116795|
|            11|  M|1142803|
|            05|  M|1144327|
|            03|  M|1231322|
|            11|  F|1142673|
|            07|  F|1088399|
|            12|  F|1242317|
|            02|  F|1177388|
|            09|  F|1065444|
|            06|  F|1066368|
|            10|  F|1144472|
|            03|  F|1262972|
+--------------+---+-------+
only showing top 20 rows



In [32]:
cdc2.groupBy('manner_of_death').count().orderBy('count', ascending=False).show()

+---------------+--------+
|manner_of_death|   count|
+---------------+--------+
|              7|19914162|
|              8| 5656790|
|              1| 1350268|
|              2|  423361|
|              3|  200528|
|              5|  119756|
|              4|   55808|
+---------------+--------+



In [ ]:
plt.hist(cdc2.manner_of_death & 50,  facecolor='g', alpha=0.75)
plt.show

In [ ]:
plt.figure()
plt.title('manner of death vs age vs sex', fontsize=18)
_ = plt.hist(cdc2.manner_of_death,20, alpha=0.9, label='M+F')
_ = plt.hist(cdc2[cdc2.sex == 'M'].age_recode_52.tolist(), 20, alpha=0.5, label='M')
_ = plt.hist(cdc2[cdc2.sex == 'F'].age_recode_52.tolist(), 20, alpha=0.5, label='F')
_ = plt.legend()

In [33]:
display(cdc2.select("sex","current_data_year").groupBy("current_data_year").agg(count("sex")).show())

+-----------------+----------+
|current_data_year|count(sex)|
+-----------------+----------+
|             2012|   2547864|
|             2014|   2631171|
|             2013|   2601452|
|             2005|   2452506|
|             2009|   2441219|
|             2006|   2430725|
|             2011|   2519842|
|             2008|   2476811|
|             2007|   2428343|
|             2015|   2718198|
|             2010|   2472542|
+-----------------+----------+



None

In [34]:
cdc2.groupBy('Place_of_death_and_decedents_status').count().orderBy('count', ascending=False).show()

+-----------------------------------+-------+
|Place_of_death_and_decedents_status|  count|
+-----------------------------------+-------+
|                                  1|9172308|
|                                  4|7504084|
|                                  6|5782818|
|                                  2|1904155|
|                                  7|1888985|
|                                  5|1287142|
|                                  3| 181181|
+-----------------------------------+-------+



In [ ]:

sqlctx =SQLContext(sc)

In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext

conf = SparkConf().setAppName("project big data")
sc = SparkContext(conf=conf)
sqlCtx = SQLContext(sc)

In [ ]:
from pyspark.sql import Row

cdc2.registerTempTable("largeTable")
display(sqlContext.sql("select * from largeTable"))

In [35]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
import pyspark.sql.functions as fn
from pyspark.sql.functions import col, udf


In [36]:
from pyspark.sql import SQLContext as sc
from pyspark.sql import Row
sqlContext = SQLContext(sc)
cdc2.registerTempTable("largeTable")
display(sqlContext.sql("select * from largeTable"))

AttributeError: type object 'SQLContext' has no attribute '_jsc'

In [ ]:
cdc2.registerTempTable("largeTable")

In [ ]:
largeTable

In [ ]:
from pyspark.sql import Row

cdc2.registerTempTable("largeTable")
sqlContext.sql()

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [ ]:
cdc2.registerTempTable("largeTable")
 MAGIC %sql Select * from largeTable

In [ ]:
import sparkmagic as MAGIC

In [ ]:
cdc2.groupBy('358_cause_recode').count().orderBy('count', ascending=False).show()

In [ ]:
cdc2.filter(cdc2.358_cause_recode == 1).count()

In [37]:
cdc = cdc2

In [38]:
cdc.printSchema()

root
 |-- resident_status: string (nullable = true)
 |-- education_2003_revision: string (nullable = true)
 |-- month_of_death: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age_recode_52: string (nullable = true)
 |-- Place_of_death_and_decedents_status: string (nullable = false)
 |-- marital_status: string (nullable = true)
 |-- day_of_week_of_death: string (nullable = true)
 |-- current_data_year: string (nullable = true)
 |-- injury_at_work: string (nullable = true)
 |-- manner_of_death: string (nullable = false)
 |-- method_of_disposition: string (nullable = true)
 |-- autopsy: string (nullable = true)
 |-- activity_code: string (nullable = false)
 |-- place_of_injury_for_causes_w00_y34_except_y06_and_y07_: string (nullable = false)
 |-- 358_cause_recode: string (nullable = true)
 |-- 113_cause_recode: string (nullable = true)
 |-- 130_infant_cause_recode: string (nullable = false)
 |-- 39_cause_recode: string (nullable = true)
 |-- number_of_entity_axis_conditi

In [39]:
cdc.where(cdc.manner_of_death == 1).groupby('manner_of_death','sex').count().show()

+---------------+---+------+
|manner_of_death|sex| count|
+---------------+---+------+
|              1|  M|872090|
|              1|  F|478178|
+---------------+---+------+



In [40]:
cdc.where(cdc.manner_of_death == 2).groupby('manner_of_death','sex').count().show()

+---------------+---+------+
|manner_of_death|sex| count|
+---------------+---+------+
|              2|  F| 91819|
|              2|  M|331542|
+---------------+---+------+



In [43]:
cdc.where(cdc.sex == 'M').groupby('manner_of_death').count().orderBy('count',ascending=False).show()

+---------------+-------+
|manner_of_death|  count|
+---------------+-------+
|              7|9711177|
|              8|2678946|
|              1| 872090|
|              2| 331542|
|              3| 158757|
|              5|  72076|
|              4|  35209|
+---------------+-------+



In [52]:
cdc.where((cdc.manner_of_death==2) & (cdc.sex=='M')).groupby('race_recode_3','sex').count().orderBy('count',ascending=False).show()

+-------------+---+------+
|race_recode_3|sex| count|
+-------------+---+------+
|            1|  M|299948|
|            3|  M| 19859|
|            2|  M| 11735|
+-------------+---+------+



In [53]:
cdc.where((cdc.manner_of_death==2) & (cdc.sex=='F')).groupby('race_recode_3','sex').count().orderBy('count',ascending=False).show()

+-------------+---+-----+
|race_recode_3|sex|count|
+-------------+---+-----+
|            1|  F|82652|
|            2|  F| 4682|
|            3|  F| 4485|
+-------------+---+-----+



In [54]:
cdc.where((cdc.manner_of_death==3) & (cdc.sex=='M')).groupby('race_recode_3','sex').count().orderBy('count',ascending=False).show()

+-------------+---+-----+
|race_recode_3|sex|count|
+-------------+---+-----+
|            3|  M|81232|
|            1|  M|72238|
|            2|  M| 5287|
+-------------+---+-----+



In [67]:
#deaths due to heart disease, decreasing trend
cdc.where((cdc.death_cause== 215)| (cdc.death_cause==211)).groupby('current_data_year').count().orderBy('current_data_year',ascending=True).show()

+-----------------+------+
|current_data_year| count|
+-----------------+------+
|             2005|379968|
|             2006|361042|
|             2007|343807|
|             2008|343078|
|             2009|325767|
|             2010|318397|
|             2011|312899|
|             2012|309038|
|             2013|307286|
|             2014|300958|
|             2015|302695|
+-----------------+------+



In [69]:
cdc.where((cdc.death_cause== 215)| (cdc.death_cause==211)).groupby('current_data_year','age_recode_52').count().orderBy('c',ascending=True).show()

AnalysisException: "cannot resolve '`current_data_year`' given input columns: [age_recode_52, count];;\n'Sort ['current_data_year ASC NULLS FIRST], true\n+- Aggregate [age_recode_52#21], [age_recode_52#21, count(1) AS count#2977L]\n   +- Filter ((cast(death_cause#2609 as int) = 215) || (cast(death_cause#2609 as int) = 211))\n      +- Project [resident_status#12, education_2003_revision#1757, month_of_death#16, sex#17, age_recode_52#21, Place_of_death_and_decedents_status#943, marital_status#26, day_of_week_of_death#27, current_data_year#28, injury_at_work#29, manner_of_death#624, method_of_disposition#863, autopsy#32, activity_code#545, place_of_injury_for_causes_w00_y34_except_y06_and_y07_#387, 358_cause_recode#36 AS death_cause#2609, 113_cause_recode#37, 130_infant_cause_recode#466, 39_cause_recode#39, number_of_entity_axis_conditions#40, number_of_record_axis_conditions#61, race#82, race_recode_3#85, race_recode_5#86, hispanic_originrace_recode#88]\n         +- Project [resident_sta

In [55]:
cdc = cdc.withColumnRenamed('358_cause_recode', 'death_cause')

In [62]:
cdc.select('death_cause').distinct().show()

+-----------+
|death_cause|
+-----------+
|        296|
|        125|
|        451|
|        124|
|        447|
|        169|
|        334|
|        030|
|        448|
|        088|
|        282|
|        232|
|        155|
|        132|
|        154|
|        317|
|        428|
|        388|
|        101|
|        009|
+-----------+
only showing top 20 rows



In [71]:
cdc.where((cdc.death_cause== 215)| (cdc.death_cause==211)).groupby('age_recode_52').count().orderBy('count',ascending=False).show()

+-------------+------+
|age_recode_52| count|
+-------------+------+
|           43|646646|
|           42|574964|
|           44|492527|
|           41|426542|
|           40|321185|
|           39|264313|
|           38|221769|
|           45|202944|
|           37|169510|
|           36|116590|
|           35| 67072|
|           46| 41248|
|           34| 32358|
|           33| 13489|
|           32|  5896|
|           47|  3560|
|           31|  2468|
|           30|   961|
|           29|   279|
|           48|   134|
+-------------+------+
only showing top 20 rows



In [79]:
cdc.where(((cdc.death_cause== 215)| (cdc.death_cause==211)) & (cdc.age_recode_52 ==30).groupby('current_data_year').count().orderBy('count',ascending=False).show()

SyntaxError: unexpected EOF while parsing (<ipython-input-79-226135a8e498>, line 1)

In [83]:
cdc.where(((cdc.death_cause== 215)| (cdc.death_cause==211)) & (cdc.age_recode_52 ==30)).groupby('current_data_year').count().orderBy('current_data_year',ascending=True).show()

+-----------------+-----+
|current_data_year|count|
+-----------------+-----+
|             2005|  107|
|             2006|   78|
|             2007|   98|
|             2008|   99|
|             2009|   70|
|             2010|   79|
|             2011|   90|
|             2012|   96|
|             2013|   93|
|             2014|   71|
|             2015|   80|
+-----------------+-----+



In [84]:
cdc.where(((cdc.death_cause== 215)| (cdc.death_cause==211)) & (cdc.age_recode_52 ==34)).groupby('current_data_year').count().orderBy('current_data_year',ascending=True).show()

+-----------------+-----+
|current_data_year|count|
+-----------------+-----+
|             2005| 3881|
|             2006| 3742|
|             2007| 3365|
|             2008| 3202|
|             2009| 2879|
|             2010| 2706|
|             2011| 2686|
|             2012| 2610|
|             2013| 2553|
|             2014| 2396|
|             2015| 2338|
+-----------------+-----+



In [85]:
#increasing breathing/lung problem deaths
cdc.where((cdc.death_cause== 267)| (cdc.death_cause=='093')).groupby('current_data_year').count().orderBy('current_data_year',ascending=True).show()

+-----------------+------+
|current_data_year| count|
+-----------------+------+
|             2005|270671|
|             2006|265551|
|             2007|269086|
|             2008|282411|
|             2009|279803|
|             2010|281490|
|             2011|285837|
|             2012|287577|
|             2013|292131|
|             2014|290276|
|             2015|296548|
+-----------------+------+



In [86]:
cdc.where((cdc.death_cause== 267)| (cdc.death_cause=='093')).groupby('age_recode_52').count().orderBy('count',ascending=False).show()

+-------------+------+
|age_recode_52| count|
+-------------+------+
|           41|506677|
|           42|490341|
|           40|465131|
|           39|391883|
|           43|357255|
|           38|289686|
|           37|194360|
|           44|167217|
|           36|113931|
|           35| 49923|
|           45| 45603|
|           34| 16012|
|           46|  5837|
|           33|  4427|
|           32|  1470|
|           31|   581|
|           47|   356|
|           30|   258|
|           29|   124|
|           52|    66|
+-------------+------+
only showing top 20 rows



In [92]:
cdc.where((cdc.death_cause== 267)| (cdc.death_cause== '093') & ((cdc.age_recode_52 ==34)|(cdc.age_recode_52 ==30)|(cdc.age_recode_52 ==31)|(cdc.age_recode_52 ==33)|(cdc.age_recode_52 ==32)).groupby('current_data_year').count().orderBy('current_data_year',ascending=True).show()

SyntaxError: unexpected EOF while parsing (<ipython-input-92-85daa141482b>, line 1)

In [93]:
#year wise, breathing/lung diseases, age 20-45
cdc.where(((cdc.death_cause== 267)| (cdc.death_cause== '093') )& ((cdc.age_recode_52 ==34)|(cdc.age_recode_52 ==30)|(cdc.age_recode_52 ==31)|(cdc.age_recode_52 ==33)|(cdc.age_recode_52 ==32))).groupby('current_data_year').count().orderBy('current_data_year',ascending=True).show()

+-----------------+-----+
|current_data_year|count|
+-----------------+-----+
|             2005| 2928|
|             2006| 2613|
|             2007| 2450|
|             2008| 2250|
|             2009| 2101|
|             2010| 1924|
|             2011| 1829|
|             2012| 1766|
|             2013| 1702|
|             2014| 1665|
|             2015| 1520|
+-----------------+-----+



In [94]:
#year wise,  early 40s age
cdc.where(((cdc.death_cause== 267)| (cdc.death_cause== '093') )& ((cdc.age_recode_52 ==34))).groupby('current_data_year').count().orderBy('current_data_year',ascending=True).show()

+-----------------+-----+
|current_data_year|count|
+-----------------+-----+
|             2005| 2209|
|             2006| 1903|
|             2007| 1812|
|             2008| 1594|
|             2009| 1448|
|             2010| 1278|
|             2011| 1297|
|             2012| 1200|
|             2013| 1157|
|             2014| 1106|
|             2015| 1008|
+-----------------+-----+



In [95]:
#age wise distribution for breathing/lung problems
cdc.where((cdc.death_cause== 267)| (cdc.death_cause=='093')).groupby('age_recode_52').count().orderBy('count',ascending=False).show()

+-------------+------+
|age_recode_52| count|
+-------------+------+
|           41|506677|
|           42|490341|
|           40|465131|
|           39|391883|
|           43|357255|
|           38|289686|
|           37|194360|
|           44|167217|
|           36|113931|
|           35| 49923|
|           45| 45603|
|           34| 16012|
|           46|  5837|
|           33|  4427|
|           32|  1470|
|           31|   581|
|           47|   356|
|           30|   258|
|           29|   124|
|           52|    66|
+-------------+------+
only showing top 20 rows



In [96]:
#deaths due to breathing/lung problems in middle age
cdc.where(((cdc.death_cause== 267)| (cdc.death_cause== '093') )& ((cdc.age_recode_52 ==35)|(cdc.age_recode_52 ==36))).groupby('current_data_year').count().orderBy('current_data_year',ascending=True).show()

+-----------------+-----+
|current_data_year|count|
+-----------------+-----+
|             2005|15534|
|             2006|15478|
|             2007|15580|
|             2008|15888|
|             2009|16134|
|             2010|15535|
|             2011|15203|
|             2012|14722|
|             2013|14177|
|             2014|13216|
|             2015|12387|
+-----------------+-----+



In [97]:
#gun deaths year wise
cdc.where((cdc.death_cause== 407)| (cdc.death_cause== 429)|(cdc.death_cause==435)|(cdc.death_cause==446) ).groupby('current_data_year').count().orderBy('current_data_year',ascending=True).show()

+-----------------+-----+
|current_data_year|count|
+-----------------+-----+
|             2005|30426|
|             2006|30612|
|             2007|30964|
|             2008|31335|
|             2009|31073|
|             2010|31382|
|             2011|31961|
|             2012|33158|
|             2013|33208|
|             2014|33184|
|             2015|35856|
+-----------------+-----+



In [98]:
#age wise distribution due to gundeaths
cdc.where((cdc.death_cause== 407)| (cdc.death_cause== 429)|(cdc.death_cause==435)|(cdc.death_cause==446) ).groupby('age_recode_52').count().orderBy('count',ascending=False).show()

+-------------+-----+
|age_recode_52|count|
+-------------+-----+
|           30|44822|
|           31|38016|
|           32|31021|
|           36|28438|
|           35|28170|
|           33|27119|
|           34|26895|
|           29|26360|
|           37|24483|
|           38|18791|
|           39|14502|
|           40|12049|
|           41|10686|
|           42| 9115|
|           43| 5671|
|           28| 2774|
|           44| 2063|
|           27|  776|
|           45|  362|
|           25|  239|
+-------------+-----+
only showing top 20 rows



In [100]:
#death reasons for black women, diabetes and breast cancer are more prominent than white women
cdc.where((cdc.race_recode_3==3)& (cdc.sex=='F')).groupby('death_cause').count().orderBy('count',ascending=False).show()

+-----------+------+
|death_cause| count|
+-----------+------+
|        215|102876|
|        159| 75507|
|        093| 75255|
|        211| 72242|
|        104| 66158|
|        175| 65174|
|        238| 51556|
|        327| 48234|
|        189| 43663|
|        207| 40438|
|        214| 39733|
|        267| 38302|
|        023| 37545|
|        230| 32148|
|        081| 32051|
|        228| 30292|
|        257| 27523|
|        088| 25514|
|        125| 23593|
|        236| 21273|
+-----------+------+
only showing top 20 rows



In [101]:
#trend for diabetes deaths for black women
cdc.where((cdc.race_recode_3==3)& (cdc.sex=='F') & (cdc.death_cause==159)).groupby('current_data_year').count().orderBy('current_data_year',ascending=True).show()

+-----------------+-----+
|current_data_year|count|
+-----------------+-----+
|             2005| 7255|
|             2006| 7046|
|             2007| 6969|
|             2008| 6613|
|             2009| 6476|
|             2010| 6490|
|             2011| 6854|
|             2012| 6843|
|             2013| 7011|
|             2014| 6987|
|             2015| 6963|
+-----------------+-----+



In [104]:
#death causes for white females
cdc.where((cdc.sex=='F') & (cdc.race_recode_3==1)).groupby('death_cause').count().orderBy('count',ascending=False).show()

+-----------+------+
|death_cause| count|
+-----------+------+
|        215|908273|
|        175|717872|
|        093|678984|
|        267|671492|
|        189|590448|
|        211|537624|
|        238|401899|
|        104|372307|
|        230|316915|
|        159|300936|
|        257|253884|
|        228|235299|
|        214|226827|
|        327|194444|
|        081|192684|
|        125|178031|
|        023|172331|
|        088|168772|
|        207|148830|
|        239|142800|
+-----------+------+
only showing top 20 rows



In [105]:
#year wise distribution for breast cancer deaths
cdc.where((cdc.sex=='F') & (cdc.death_cause==104)).groupby('current_data_year').count().orderBy('current_data_year',ascending=True).show()

+-----------------+-----+
|current_data_year|count|
+-----------------+-----+
|             2005|41170|
|             2006|40866|
|             2007|40647|
|             2008|40639|
|             2009|40716|
|             2010|41037|
|             2011|40967|
|             2012|41206|
|             2013|40906|
|             2014|41262|
|             2015|41566|
+-----------------+-----+



In [106]:
#age distribution for breast cancer deaths
cdc.where((cdc.sex=='F') & (cdc.death_cause==104)).groupby('age_recode_52').count().orderBy('count',ascending=False).show()

+-------------+-----+
|age_recode_52|count|
+-------------+-----+
|           38|51082|
|           39|49395|
|           42|47214|
|           37|46897|
|           41|46390|
|           40|46030|
|           43|39893|
|           36|37807|
|           35|25611|
|           44|23619|
|           34|15505|
|           45| 8302|
|           33| 7678|
|           32| 3059|
|           46| 1441|
|           31|  808|
|           47|  118|
|           30|  112|
|           52|   10|
|           29|    4|
+-------------+-----+
only showing top 20 rows



In [111]:
#breast cancer death trend in middle aged women
cdc.where((cdc.death_cause==104) & ((cdc.age_recode_52==33)|(cdc.age_recode_52==34)|(cdc.age_recode_52==35))).groupby('current_data_year').count().orderBy('current_data_year',ascending=True).show()


+-----------------+-----+
|current_data_year|count|
+-----------------+-----+
|             2005| 5159|
|             2006| 5004|
|             2007| 4702|
|             2008| 4656|
|             2009| 4681|
|             2010| 4377|
|             2011| 4226|
|             2012| 4321|
|             2013| 4101|
|             2014| 3992|
|             2015| 3855|
+-----------------+-----+



In [113]:
#all drugs related deaths combined(includes suicide, accident and medicinal overdose)
cdc.where((cdc.death_cause==420) | (cdc.death_cause==433) | (cdc.death_cause==425) | (cdc.death_cause==443) | (cdc.death_cause==454)).groupby('current_data_year').count().orderBy('current_data_year',ascending=True).show()


+-----------------+-----+
|current_data_year|count|
+-----------------+-----+
|             2005|30274|
|             2006|34896|
|             2007|36474|
|             2008|37028|
|             2009|37521|
|             2010|38850|
|             2011|41901|
|             2012|42054|
|             2013|44509|
|             2014|47599|
|             2015|53096|
+-----------------+-----+



In [114]:
#medicine/drugs(therapeutic use) trend
cdc.where((cdc.death_cause==454)).groupby('current_data_year').count().orderBy('current_data_year',ascending=True).show()


+-----------------+-----+
|current_data_year|count|
+-----------------+-----+
|             2005|  344|
|             2006|  349|
|             2007|  351|
|             2008|  432|
|             2009|  407|
|             2010|  427|
|             2011|  414|
|             2012|  408|
|             2013|  385|
|             2014|  384|
|             2015|  455|
+-----------------+-----+



In [115]:
#Drug overdose deaths increasing
cdc.where((cdc.death_cause==420) | (cdc.death_cause==443)).groupby('current_data_year').count().orderBy('current_data_year',ascending=True).show()


+-----------------+-----+
|current_data_year|count|
+-----------------+-----+
|             2005|25594|
|             2006|29862|
|             2007|31256|
|             2008|31461|
|             2009|31974|
|             2010|33038|
|             2011|36068|
|             2012|36066|
|             2013|38585|
|             2014|41667|
|             2015|47314|
+-----------------+-----+



In [56]:
cdc.printSchema()

root
 |-- resident_status: string (nullable = true)
 |-- education_2003_revision: string (nullable = true)
 |-- month_of_death: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age_recode_52: string (nullable = true)
 |-- Place_of_death_and_decedents_status: string (nullable = false)
 |-- marital_status: string (nullable = true)
 |-- day_of_week_of_death: string (nullable = true)
 |-- current_data_year: string (nullable = true)
 |-- injury_at_work: string (nullable = true)
 |-- manner_of_death: string (nullable = false)
 |-- method_of_disposition: string (nullable = true)
 |-- autopsy: string (nullable = true)
 |-- activity_code: string (nullable = false)
 |-- place_of_injury_for_causes_w00_y34_except_y06_and_y07_: string (nullable = false)
 |-- death_cause: string (nullable = true)
 |-- 113_cause_recode: string (nullable = true)
 |-- 130_infant_cause_recode: string (nullable = false)
 |-- 39_cause_recode: string (nullable = true)
 |-- number_of_entity_axis_conditions: 

In [ ]:
cdc2.groupBy('358_cause_recode', 'sex').count().orderBy('count', ascending=False).show()

In [ ]:
from pyspark.sql.functions import lit

In [ ]:
cdc2.groupBy('358_cause_recode'= 215, 'sex').count().orderBy('count', ascending=False).show()

In [ ]:
cdc2.groupBy('358_cause_recode', 'sex').count().orderBy('count', ascending=False).show().filter("358_cause_recode" = 407)

In [ ]:
cdc2 = cdc2.selectExpr("358_cause_recode as death_cause")
cdc2.show()
cdc2.printSchema()

In [ ]:
abc = abc.withColumnRenamed(cdc,'358_cause_recode', 'death_cause').collect()